In [ ]:
import pandas as pd

# Load the dataset using semicolon as the delimiter
file_path = "All_data.csv"
df = pd.read_csv(file_path, sep=";", encoding="utf-8")

# Save the dataset with comma as the delimiter
df.to_csv("All_data_comma.csv", sep=",", index=False, encoding="utf-8")



In [ ]:
import pandas as pd

# Load the dataset using comma as the delimiter; read all columns as strings initially
file_path = "All_data_comma.csv"  
df = pd.read_csv(file_path, delimiter=",", dtype=str)

# Convert 'volgNr' column to numeric format, allowing for missing values
df["volgNr"] = pd.to_numeric(df["volgNr"], errors='coerce').astype("Int64")

# Define a mapping from original column names to standard month abbreviations
periode_kolommen = {
    "strFormuleRealisatiePeriode01": "Jan",
    "strFormuleRealisatiePeriode02": "Feb",
    "strFormuleRealisatiePeriode03": "Mar",
    "strFormuleRealisatiePeriode04": "Apr",
    "strFormuleRealisatiePeriode05": "May",
    "strFormuleRealisatiePeriode06": "Jun",
    "strFormuleRealisatiePeriode07": "Jul",
    "strFormuleRealisatiePeriode08": "Aug",
    "strFormuleRealisatiePeriode09": "Sep",
    "strFormuleRealisatiePeriode10": "Oct",
    "strFormuleRealisatiePeriode11": "Nov",
    "strFormuleRealisatiePeriode12": "Dec"
}

# Convert all monthly value columns to float (after replacing commas with dots)
for col in periode_kolommen.keys():
    df[col] = df[col].str.replace(",", ".").astype(float)

# Reshape the dataset from wide format to long format (one row per period)
df_melted = df.melt(
    id_vars=["volgNr", "Boekjaar", "Omschrijving"],  # keep these columns fixed
    value_vars=periode_kolommen.keys(),              # columns to unpivot
    var_name="Periode",                              # new column: original column names
    value_name="Waarde"                              # new column: values
)

# Replace long period names with standard month abbreviations
df_melted["Periode"] = df_melted["Periode"].map(periode_kolommen)

# Create a proper date column combining month and year, formatted as 'YYYY-MM-DD'
df_melted["Datum"] = pd.to_datetime(
    df_melted["Periode"] + " " + df_melted["Boekjaar"].astype(str),
    format="%b %Y"
).dt.strftime('%Y-%m-%d')

# Reshape data back into wide format: one row per (farm, date), one column per metric
df_final = df_melted.pivot_table(
    index=["volgNr", "Datum"],
    columns="Omschrijving",
    values="Waarde",
    aggfunc="sum"
).reset_index()

# Convert values to numeric where possible, keep other types unchanged
df_final = df_final.apply(pd.to_numeric, errors='ignore')

# Save the final transformed dataset to a new CSV file using semicolon as delimiter
output_file = "Transformed_Corrected_All_data.csv"
df_final.to_csv(output_file, index=False, sep=";")


In [ ]:
import pandas as pd

# Load the main transformed dataset
file_transformed = "Transformed_Corrected_All_data.csv"
df_transformed = pd.read_csv(file_transformed, sep=";", encoding="utf-8")

# Load the dataset containing milk cow counts
file_melkkoeien = "melkkoeien.csv"
df_melkkoeien = pd.read_csv(file_melkkoeien, sep=";", encoding="utf-8")

# Standardize column names for consistent merging
df_melkkoeien.rename(columns={"VolgNr": "volgnr"}, inplace=True)
df_transformed.rename(columns={"volgNr": "volgnr"}, inplace=True)

# Extract the month number from the indicator name (e.g., "Aantal_Melkkoeien_03" → 03)
df_melkkoeien["Maand"] = df_melkkoeien["KengetalOms"].str.extract(r'(\d+)')
df_melkkoeien["Maand"] = pd.to_numeric(df_melkkoeien["Maand"], errors="coerce")

# Convert cow count values to float (replace comma with dot)
df_melkkoeien["Aantal"] = df_melkkoeien["Aantal"].str.replace(",", ".")
df_melkkoeien["Aantal"] = pd.to_numeric(df_melkkoeien["Aantal"], errors="coerce")

# Extract month and year from the 'Datum' column in the financial dataset
df_transformed["Maand"] = pd.to_datetime(df_transformed["Datum"]).dt.month
df_transformed["Boekjaar"] = pd.to_datetime(df_transformed["Datum"]).dt.year

# Merge the cow count data into the financial data using farm ID, year, and month
df_final = df_transformed.merge(df_melkkoeien, on=["volgnr", "Boekjaar", "Maand"], how="left")

# Rename the 'Aantal' column for clarity
df_final.rename(columns={"Aantal": "Aantal melkkoeien"}, inplace=True)

# Replace missing cow count values with 0
df_final["Aantal melkkoeien"].fillna(0, inplace=True)

# Drop columns that are no longer needed
df_final.drop(columns=["KengetalOms", "Kengetal"], inplace=True)

# Save the final merged dataset to a new CSV file
output_file = "Transformed_Corrected_With_Melkkoeien.csv"
df_final.to_csv(output_file, sep=";", index=False, encoding="utf-8")


In [ ]:
import pandas as pd

# Load the dataset containing financial data and cow counts
file_path = "Transformed_Corrected_With_Melkkoeien.csv"
df = pd.read_csv(file_path, sep=";")

# Define the names of the three cash flow components
cashflow_columns = [
    "Kasstroom uit bedrijf",       # Operating cash flow
    "Kasstroom uit financiering",  # Financing cash flow
    "Kasstroom uit investering"    # Investing cash flow
]

# Validate presence of required columns
missing_cols = [col for col in cashflow_columns if col not in df.columns]

# Calculate the total cash flow by summing the three components
df["Total Cash Flow"] = df[cashflow_columns].sum(axis=1)

# Drop the original individual cash flow columns to reduce redundancy
df.drop(columns=cashflow_columns, inplace=True)

# Export the updated dataset to a new CSV file
df.to_csv("data_cashflow.csv", sep=";", index=False, encoding="utf-8")


In [ ]:
import pandas as pd

# Step 1: Load the existing dataset that includes temperature and financial data
file_data = "data_with_temperature.csv"
df_data = pd.read_csv(file_data, sep=";", encoding="utf-8")

# Step 2: Load the monthly precipitation dataset
file_precipitation = "monthly_precipitation.csv"
df_precipitation = pd.read_csv(file_precipitation, sep=",", encoding="utf-8")

# Step 3: Transform the precipitation data into long format
df_precipitation = df_precipitation.melt(
    id_vars=["Jaar"],                  # Year column remains fixed
    var_name="Maand",                  # Original month names become values in a new column
    value_name="Precipitation (mm)"   # Precipitation values
)

# Step 4: Map month abbreviations to numerical month values
month_mapping = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}
df_precipitation["Maand"] = df_precipitation["Maand"].map(month_mapping)

# Step 5: Merge precipitation data into the main dataset based on year and month
df_final = df_data.merge(df_precipitation, on=["Jaar", "Maand"], how="left")


# Step 6: Export the final enriched dataset to CSV
output_file = "data_with_temperature_precipitation.csv"
df_final.to_csv(output_file, sep=";", index=False, encoding="utf-8")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the combined dataset including financials, livestock, temperature, and precipitation
file_path = "data_with_temperature_precipitation.csv"
df = pd.read_csv(file_path, delimiter=";")

# Standardize column names: lowercase, replace spaces and periods with underscores
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace(".", "", regex=False)
)

# Manually fix any known column name issues
df.rename(columns={"bank,_rekening-courant": "bank_rekening_courant"}, inplace=True)

# Print number of missing values per column
print("\nMissing values per column:\n", df.isnull().sum())

# Check and report number of duplicate rows
duplicates = df.duplicated().sum()
print("\nNumber of duplicate rows: ", duplicates)

# Display summary statistics for all numeric columns
print("\nDataset summary:")
print(df.describe())




In [ ]:
import pandas as pd
import numpy as np

# Load the enriched dataset with financial, climate, and livestock data
file_path = "data_with_temperature_precipitation.csv"  
df = pd.read_csv(file_path, delimiter=";", encoding="utf-8")

# Standardize column names for consistency
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace(".", "")

# Convert the date column to datetime format
df["datum"] = pd.to_datetime(df["datum"], errors="coerce")

# Extract time-based features from the date
df["maand"] = df["datum"].dt.month        # Month (1–12)
df["kwartaal"] = df["datum"].dt.quarter   # Quarter (1–4)
df["jaar"] = df["datum"].dt.year          # Year (e.g., 2021)

# Define lag periods and the target column
lag_features = [1, 3, 6, 12]  
target_col = "totale_kasstroom"  

# Create lag features (previous values of the target)
for lag in lag_features:
    df[f"{target_col}_lag{lag}"] = df[target_col].shift(lag)

# Define rolling window sizes (this was missing in your code)
rolling_windows = [3, 6, 12]

# Create rolling average features over past periods
for window in rolling_windows:
    df[f"{target_col}_rolling{window}"] = df[target_col].rolling(window=window).mean()

# Drop rows with NaNs (caused by shifting/rolling at the beginning)
df.dropna(inplace=True)

# Export the feature-enhanced dataset
output_file = "data_features.csv"
df.to_csv(output_file, sep=";", index=False, encoding="utf-8")



In [ ]:
import pandas as pd

file_path = "data_features.csv"
df = pd.read_csv(file_path, delimiter=";")

df['datum'] = pd.to_datetime(df['datum'])

missing_values_summary = df.isnull().sum()
missing_values_summary = missing_values_summary[missing_values_summary > 0]

Q1 = df['totale_kasstroom'].quantile(0.25)
Q3 = df['totale_kasstroom'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outlier_count = ((df['totale_kasstroom'] < lower_bound) | (df['totale_kasstroom'] > upper_bound)).sum()

print(f"Aantal kolommen met missende waarden: {len(missing_values_summary)}")
print(f"Aantal outliers in totale_kasstroom: {outlier_count}")

zero_threshold = 0.95
zero_percentage = (df == 0).sum() / len(df)

columns_to_drop = zero_percentage[zero_percentage > zero_threshold].index.tolist()
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Aantal verwijderde kolommen: {len(columns_to_drop)}")
print(f"Aantal overgebleven kolommen: {df_cleaned.shape[1]}")

df_cleaned.to_csv("cleaned_data.csv", index=False)

In [ ]:
import pandas as pd

# Load the dataset with engineered features
file_path = "data_features.csv"
df = pd.read_csv(file_path, delimiter=";")

# Ensure the 'datum' column is in datetime format
df['datum'] = pd.to_datetime(df['datum'])

# Identify columns with missing values
missing_values_summary = df.isnull().sum()
missing_values_summary = missing_values_summary[missing_values_summary > 0]

# Detect outliers in the 'totale_kasstroom' column using the IQR method
Q1 = df['totale_kasstroom'].quantile(0.25)
Q3 = df['totale_kasstroom'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outlier_count = ((df['totale_kasstroom'] < lower_bound) | (df['totale_kasstroom'] > upper_bound)).sum()

# Print missing values and outlier summary
print(f"Number of columns with missing values: {len(missing_values_summary)}")
print(f"Number of outliers in 'totale_kasstroom': {outlier_count}")

# Identify columns with more than 95% zero values
zero_threshold = 0.95
zero_percentage = (df == 0).sum() / len(df)

columns_to_drop = zero_percentage[zero_percentage > zero_threshold].index.tolist()

# Drop those low-information columns
df_cleaned = df.drop(columns=columns_to_drop)

# Report the result
print(f"Number of columns removed due to >95% zero values: {len(columns_to_drop)}")
print(f"Number of remaining columns: {df_cleaned.shape[1]}")

# Save the cleaned dataset
df_cleaned.to_csv("cleaned_data.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

# Load the cleaned dataset
file_path = "cleaned_data.csv"
df = pd.read_csv(file_path)

# Convert the 'datum' column to datetime and set it as the index
df['datum'] = pd.to_datetime(df['datum'])
df.set_index('datum', inplace=True)

# Extract time-based features from the datetime index
df['maand'] = df.index.month
df['kwartaal'] = df.index.quarter

# Perform one-hot encoding on 'month' and 'quarter', dropping the first category to avoid multicollinearity
df = pd.get_dummies(df, columns=['maand', 'kwartaal'], drop_first=True)

# Export the final feature-engineered dataset
df.to_csv("feature_engineered_data.csv")


In [ ]:
import pandas as pd

# Load the feature-engineered dataset
file_path = "feature_engineered_data.csv"
df = pd.read_csv(file_path)

# Define columns to remove (likely redundant or correlated features)
columns_to_remove = [
    "mutatie_liquide_middelen_vlgs_opstelling",
    "mutatie_liq_middelen_vlgs_administratie",
    "liquide_middelen",
    "bank,_rekening-courant"
]

# Remove specified columns; ignore errors if any column is missing
df_cleaned = df.drop(columns=columns_to_remove, errors='ignore')

# Save the final cleaned dataset
df_cleaned.to_csv("cleaned_feature_data.csv", index=False)


In [ ]:
import pandas as pd

# Load the feature-engineered dataset
df = pd.read_csv("feature_engineered_data.csv")

# Define the target variable
target_column = "totale_kasstroom"

# Select only numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=["number"]).copy()

# Ensure rows with missing target values are excluded
numeric_df = numeric_df[numeric_df[target_column].notna()]

# Compute the absolute correlation matrix
corr_matrix = numeric_df.corr().abs()

# Extract correlation values with the target (excluding the target itself)
target_corr = corr_matrix[target_column].drop(target_column)

# Define a correlation threshold for multicollinearity
threshold = 0.9
to_remove = set()

# Iterate over upper triangle of the correlation matrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]

        # Skip if either column is the target
        if col1 == target_column or col2 == target_column:
            continue

        # If correlation between two features is too high, remove the less informative one
        if corr_matrix.loc[col1, col2] > threshold:
            corr1 = abs(corr_matrix.loc[col1, target_column])
            corr2 = abs(corr_matrix.loc[col2, target_column])

            if corr1 >= corr2:
                to_remove.add(col2)
            else:
                to_remove.add(col1)

# Drop highly collinear features from the original dataframe
df_cleaned = df.drop(columns=to_remove)

# Export the final dataset with reduced multicollinearity
df_cleaned.to_csv("data_no_multicorr.csv", index=False)

# Print summary
print(f"Number of removed features due to high correlation: {len(to_remove)}")
print(f"Number of remaining features: {df_cleaned.shape[1]}")


In [ ]:
import pandas as pd

# 1. Load the dataset without multicollinearity issues
df = pd.read_csv("data_no_multicorr.csv")

# 2. Convert the 'datum' column to datetime and sort by farmer and time
df["datum"] = pd.to_datetime(df["datum"])
df = df.sort_values(["volgnr", "datum"]).reset_index(drop=True)

# 3. Function to add lagged and rolling features for each individual farm
def add_lagged_features(group):
    group = group.copy()
    group = group.sort_values("datum")

    # Lagged features: previous values
    group["kasstroom_lag1"] = group["totale_kasstroom"].shift(1)
    group["kasstroom_lag3"] = group["totale_kasstroom"].shift(3)

    # Rolling averages (with shift to avoid data leakage)
    group["kasstroom_rolling3"] = group["totale_kasstroom"].shift(1).rolling(3).mean()
    group["kasstroom_rolling6"] = group["totale_kasstroom"].shift(1).rolling(6).mean()

    return group

# 4. Apply the feature engineering per individual farmer (volgnr)
df_with_lags = df.groupby("volgnr").apply(add_lagged_features).reset_index(drop=True)

# 5. Save the enriched dataset
df_with_lags.to_csv("data_met_lagged_features.csv", index=False)


In [ ]:
import pandas as pd

# Load the dataset with lagged features
df = pd.read_csv("data_met_lagged_features.csv")

# Ensure the date column is parsed correctly
df['datum'] = pd.to_datetime(df['datum'])

# Create a 'year-month' column for monthly grouping
df['jaar_maand'] = df['datum'].dt.to_period('M')

# Define realistic price boundaries (€/kg)
min_prijs = 0       # No negative prices allowed
max_prijs = 5       # Maximum realistic price threshold

# Identify outliers for milk and feed price
melkprijs_outliers = (df['melkprijs_per_kg'] < min_prijs) | (df['melkprijs_per_kg'] > max_prijs)
voerprijs_outliers = (df['voerprijs'] < min_prijs) | (df['voerprijs'] > max_prijs)

# Calculate monthly averages using only valid values
melkprijs_gemiddelden = df.loc[~melkprijs_outliers].groupby('jaar_maand')['melkprijs_per_kg'].mean()
voerprijs_gemiddelden = df.loc[~voerprijs_outliers].groupby('jaar_maand')['voerprijs'].mean()

# Replace outliers with the monthly average for that month
df.loc[melkprijs_outliers, 'melkprijs_per_kg'] = df.loc[melkprijs_outliers, 'jaar_maand'].map(melkprijs_gemiddelden)
df.loc[voerprijs_outliers, 'voerprijs'] = df.loc[voerprijs_outliers, 'jaar_maand'].map(voerprijs_gemiddelden)

# Remove the temporary grouping column
df.drop(columns='jaar_maand', inplace=True)

# Save the cleaned dataset
df.to_csv("data_corrected_melk_voerprijzen.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Step 1: Load the cleaned dataset with milk and feed prices
df = pd.read_csv("data_corrected_melk_voerprijzen.csv", parse_dates=["datum"])
df['datum'] = pd.to_datetime(df['datum'])  # Ensure date is properly parsed

# Step 2: Aggregate farm-level business characteristics
df_agg = df.groupby('volgnr').agg({
    'melkprijs_per_kg': 'mean',         # Average milk price
    'voerprijs': 'mean',                # Average feed price
    'aankopen_rundvee': 'sum',          # Total cattle purchases
    'aanwas_rundvee': 'sum',            # Total cattle growth
    'totale_kasstroom': ['mean', 'std'] # Mean and variability of cash flow
}).dropna()

# Flatten multi-level column names
df_agg.columns = ['_'.join(col) for col in df_agg.columns]

# Step 3: Normalize features and apply KMeans clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_agg)

kmeans_bedrijf = KMeans(n_clusters=3, random_state=42)
df_agg['bedrijf_cluster'] = kmeans_bedrijf.fit_predict(X_scaled)

# Step 4: Merge cluster labels back into the original dataset
df = df.merge(df_agg[['bedrijf_cluster']], left_on='volgnr', right_index=True, how='left')

# Step 5: Save the dataset with cluster labels
df.to_csv("data_with_clusters.csv", index=False)



In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("data_with_clusters.csv")

# Ensure date column is parsed
df['datum'] = pd.to_datetime(df['datum'], errors='coerce')

# Identify invalid entries (zero cows)
invalid_mask = df['aantal_melkkoeien'] == 0

# Step 1: Compute the average number of cows per farm (excluding zeros)
bedrijfsgemiddelde = df.loc[~invalid_mask].groupby('volgnr')['aantal_melkkoeien'].mean()

# Step 2: Create new column to track imputations
df['koeien_geimputeerd'] = False

# Step 3: Impute zero values with the farm average
impute_mask = invalid_mask & df['volgnr'].isin(bedrijfsgemiddelde.index)
df.loc[impute_mask, 'aantal_melkkoeien'] = df.loc[impute_mask, 'volgnr'].map(bedrijfsgemiddelde)
df.loc[impute_mask, 'koeien_geimputeerd'] = True

# Step 4: Impute remaining (NaNs) with global fallback mean of farm averages
df['aantal_melkkoeien'] = df['aantal_melkkoeien'].fillna(bedrijfsgemiddelde.mean())
df.loc[df['aantal_melkkoeien'] == bedrijfsgemiddelde.mean(), 'koeien_geimputeerd'] = True

# Save the cleaned dataset
df.to_csv("data_with_clusters.csv", index=False)



In [ ]:
import pandas as pd

# Step 1: Load and sort the dataset
df = pd.read_csv("data_with_clusters.csv")
df["datum"] = pd.to_datetime(df["datum"])
df = df.sort_values(["volgnr", "datum"]).reset_index(drop=True)

# Step 2: Define variables and their Granger-relevant lags
granger_lags = {
    "totale_kasstroom": [1, 3],
    "melkprijs_per_kg": [1, 3, 6],
    "totaal_opbrengsten": [1, 3, 6],
    "krachtvoerkosten": [1, 3, 6],
    "voerkosten": [4, 5, 6],
    "gemiddelde_temperatuur": [5, 6],
}

# Step 3: Function to apply lag features without leakage (per farm)
def add_granger_lags(group):
    group = group.sort_values("datum").copy()
    
    for var, lags in granger_lags.items():
        if var in group.columns:
            for lag in lags:
                colname = f"{var}_lag_{lag}"
                group[colname] = group[var].shift(lag)
    
    return group

# Step 4: Apply the lag function per farmer ('volgnr')
df_lagged = df.groupby("volgnr").apply(add_granger_lags).reset_index(drop=True)

df_lagged.to_csv("data_met_granger_lags.csv", index=False)


In [ ]:
import pandas as pd

# Load the dataset with winsorized financial features
df = pd.read_csv("data_met_granger_lags.csv")

# Define columns where negative values are invalid and should be corrected
cols_to_fix = ["voerkosten", "krachtvoerkosten", "totaal_opbrengsten"]

# Correct negative values by taking the absolute value
for col in cols_to_fix:
    df[col] = df[col].abs()

df.to_csv("data_final.csv", index=False)
